In [45]:
import os
root = os.getcwd().rsplit('/',1)[0]
vocab_path = '{0:s}/etc/vocab.ttl'.format(root)
dict_path = '{0:s}/etc/dict.ttl'.format(root)
print(vocab_path)
print(dict_path)

/global/u1/s/sleak/Monitoring/Resilience/LogSet/etc/vocab.ttl
/global/u1/s/sleak/Monitoring/Resilience/LogSet/etc/dict.ttl


In [46]:
import sys
sys.path.append('/global/homes/s/sleak/.local/cori/3.6-anaconda-4.4/lib/python3.6/site-packages')
import rdflib
#graph = rdflib.ConjunctiveGraph().parse(vocab_path, format='turtle')
gvocab = rdflib.ConjunctiveGraph().parse(vocab_path, format='turtle')

# now add the data dictionary:
#graph.parse(dict_path, format='turtle')
gdict = rdflib.ConjunctiveGraph().parse(dict_path, format='turtle')
graph = gvocab + gdict

print(graph.serialize(format='n3').decode('ascii'))
#print(graph.identifier)
#print(gvocab.identifier)
#print(gdict.identifier)



@prefix : <https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#> .
@prefix adms: <http://www.w3.org/ns/adms#> .
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix default1: <https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix logset: <https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:console_logfile a logset:LogSeries ;
    logset:logFormat :timeStampedLogFile ;
    logset:logFormatInfo "filepattern=console",
        "filepattern=console-<dateYYYYMMDD>",
        "part_word=1",
        "ts_word=0" .

:messages_logfile a logset:LogSeries ;
    logset:logFormat :timeStampedLogFile 

In [47]:
gindex = rdflib.ConjunctiveGraph().parse('{0:s}/examples/index.ttl'.format(root), format='turtle')
#graph.parse('{0:s}/examples/index.ttl'.format(root), format='turtle')
graph = (gvocab + gdict) + gindex
#print(graph.serialize(format='n3').decode('ascii'))
#for s,p,o in graph:
for s,p,o in gindex:    
    print ((s,p,o))
    

(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b1'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#LogSet'))
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b2'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#isLogOf'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#console_logfile'))
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b3'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#path'), rdflib.term.Literal('p0-20170906t151820/console-20170907'))
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b7'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#File'))
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b1'), rdflib.term.URIRef('https://

In [48]:
query = '''SELECT ?id ?desc
WHERE {
?id a logset:LogSet .
?id dct:description ?desc .
}'''
#print(query[21])
for row in graph.query(query):
    print(row)


(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b1'), rdflib.term.Literal('a partial sample of cori log data for testing logset tools'))


In [49]:
query = '''SELECT ?id ?path
WHERE {
?id logset:isLogOf dict:console_logfile .
?id logset:path ?path .
} '''
print("quering the whole graph:")
for row in graph.query(query):
    print(row)
    
# if I don't explicitly include the logset.ttl in the graph:
print("querying only gindex:")
# (oh - actually everything needed is in the index.ttl)
for row in gindex.query(query):
    print(row)


quering the whole graph:
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b2'), rdflib.term.Literal('p0-20170906t151820/console-20170906'))
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b3'), rdflib.term.Literal('p0-20170906t151820/console-20170907'))
querying only gindex:
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b2'), rdflib.term.Literal('p0-20170906t151820/console-20170906'))
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b3'), rdflib.term.Literal('p0-20170906t151820/console-20170907'))


In [50]:
query = '''SELECT ?id ?path
WHERE {
?id a ?type .
?type rdfs:subClassOf* logset:DataSource . 
OPTIONAL { ?id logset:path ?path }
} '''
# :DataSource is defined in vocab.ttl .. so if our graph includes that (local file) we get:
print ("with full graph:")
for row in graph.query(query):
    print(row)
    
# on the other hand, if we only query the local gindex graph:
print ("with gindex only:")
for row in gindex.query(query):
    print(row)


with full graph:
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b2'), rdflib.term.Literal('p0-20170906t151820/console-20170906'))
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b7'), rdflib.term.Literal('p0-20170906t151820/messages-20170907'))
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b6'), rdflib.term.Literal('p0-20170906t151820/messages-20170906'))
(rdflib.term.BNode('fb7c7d5ab1a1e46259948b6d277357ec5b3'), rdflib.term.Literal('p0-20170906t151820/console-20170907'))
with gindex only:


In [51]:
# so to assemble the full graph, I need to explicitly parse the vocab and data dictionary:
# (mostly useful for catalog, as it will find subgraphs from there)

# can i parse a remote graph at all?
#g1 = rdflib.Graph().parse('https://www.w3.org/ns/dcat.ttl')
#print (g1)
# yes! but providing a format might be necessary
#gg = rdflib.Graph().parse('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab.ttl', format='turtle')
gindex = rdflib.ConjunctiveGraph().parse('{0:s}/examples/index.ttl'.format(root), format='turtle')
##for s,p,o in gindex:    
##    print ((s,p,o))
for n in gindex.namespaces():
    print (n)
#print(dir(rdflib.term.URIRef))
#print(rdflib.term.URIRef.__class__)
g = gindex
print ("g now has {0} statements".format(len(g)))
for uri in [n[1] for n in gindex.namespaces() if n[0] in ('logset','dict')]:
    print(str(uri.n3()))
    print(str(uri)[:-1])
    gsource=str(uri)[:-1] + '.ttl'   # needs extension or 404!
    g += rdflib.ConjunctiveGraph().parse(gsource, format='turtle')
    print ("g now has {0} statements".format(len(g)))
    #summary = g.serialize(format='n3').decode('ascii')
    #print(summary.splitlines()[:10])
    #print(g.serialize(format='n3').decode('ascii'))

('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace'))
('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#'))
('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#'))
('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#'))
('logset', rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#'))
('dict', rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#'))
('entities', rdflib.term.URIRef('http://portal.nersc.gov/project/m888/resilience/entities/#'))
('dct', rdflib.term.URIRef('http://purl.org/dc/terms/'))
('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#'))
g now has 24 statements
<https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#>
https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab
g now has 83 statements
<https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#>
https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict
g now ha

In [53]:
query = '''SELECT ?id ?path
WHERE {
?id logset:isLogOf dict:console_logfile .
?id logset:path ?path .
} '''
# now we can find the datasources in the loaded graph
query2 = '''SELECT ?id ?path
WHERE {
?id a ?type .
?type rdfs:subClassOf* logset:DataSource . 
OPTIONAL { ?id logset:path ?path }
} '''
#query3 = '''SELECT ?id ?type ?base
#WHERE {
#?id a ?type .
#OPTIONAL { ?type rdfs:subClassOf ?base }
#}'''
for row in g.query(query2):
    print(row)

(rdflib.term.BNode('f35ae0a07a6b846599fddf8c4eaeaa924b3'), rdflib.term.Literal('p0-20170906t151820/console-20170907'))
(rdflib.term.BNode('f35ae0a07a6b846599fddf8c4eaeaa924b7'), rdflib.term.Literal('p0-20170906t151820/messages-20170907'))
(rdflib.term.BNode('f35ae0a07a6b846599fddf8c4eaeaa924b2'), rdflib.term.Literal('p0-20170906t151820/console-20170906'))
(rdflib.term.BNode('f35ae0a07a6b846599fddf8c4eaeaa924b6'), rdflib.term.Literal('p0-20170906t151820/messages-20170906'))
